In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
filters = {
    "item_conditions": {
        "New": 1000,
        "Open box": 1500,
        "Used": 3000,
        "Certified Refurbished": 2000,
        "Excellent - Refurbished": 2500,
        "Very Good": 3000,
        "Good": 4000,
        "For Parts or Not Working": 7000
    },
    "item_locations": {
        "Domestic": 1,
        "International": 2,
        "Continent": 3,
    },
    "directories": {
        "No Directory": 0,
        "Consumer Electronics": 9355,
        "Clothing, Shoes & Accessories": 11450,
        "Health & Beauty": 26395,
        "Home & Garden": 11700,
        "Sporting Goods": 382,
        "Toys & Hobbies": 220,
        "Books": 267,
        "Video Games & Consoles": 1249,
        "Collectibles": 1,
        "Business & Industrial": 12576,
        "Automotive": 6000, 
    },
    "categories": {
        "No Category": 0,
        "Cell Phones & Smartphones": 9355,
        "Laptops & Netbooks": 175673,
        "Watches": 31387,
        "Furniture": 3197,
        "Action Figures": 2605,
        "Jewelry & Watches": 281,
        "Cameras & Photo": 625,
        "Pet Supplies": 1281,
        "Crafts": 14339,
        "Computers/Tablets & Networking": 58058,
        "Cars & Trucks": 6001,  
        "Motorcycles": 6024,  
        "Car & Truck Parts": 6030,  
        "Motorcycle Parts": 10063,  
        "Automotive Tools & Supplies": 34998,  
    },
    "sort_order": {
        "Best Match": 12,
        "Time: ending soonest": 1,
        "Time: newly listed": 10,
        "Price + Shipping: lowest first": 15,
        "Price + Shipping: highest first": 16,
        "Distance: nearest first": 7
    }
}

In [6]:
# Define the base URL for the eBay search
url = "https://www.ebay.com/sch/i.html"

# Define the query parameters for the search request
params = {
    '_from': 'R40',
    '_nkw': 'iphone',
    'Brand': 'Apple',
    'LH_ItemCondition': filters["item_conditions"]["Used"],  # Item condition; 'New'.
    'LH_PrefLoc': filters["item_locations"]["Domestic"],
    '_udlo': '200',  # Minimum price.
    '_udhi': '400',  # Maximum price.
    '_dcat': filters["directories"]["No Directory"],  # Filter by directory ID; "Consumer Electronics".
    '_sacat': filters["categories"]["No Category"],  # Filter by category ID; "Cell Phones & Smartphones".
    '_sop': filters["sort_order"]["Time: newly listed"],  # Sort by "Time: newly listed"
    'LH_Sold': '0',  # Only sold listings (='1').
    #'LH_Complete': '1',  # Only completed listings (='1').
    'LH_BIN': '0',  # Only Buy It Now listings (='1').
    'LH_Auction': '1',  # Only Auction Listings (='1').
    'LH_BO': '0',  # Only listings that accept offers (='1').
    'LH_FS': '0',  # Only Free Shipping listings (='1').
    '_ipg': '240',  # Number of items per page (='1'), Max is 240.
    'rt': 'nc'  # Result type; 'nc' indicates no cache to ensure the search results are fresh. 
    
}

In [8]:
# Create a prepared request to inspect the full URL
request  =requests.Request('GET', url, params=params)
prepared_request = request.prepare()
print(prepared_request.url)

https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc


In [10]:
# Initialize variables
page_number = 0
items_list = []

# Loop over pages
while True:
    
    page_number += 1
    
    print(f'Scraping page: {page_number}')
    request  =requests.Request('GET', url, params=params)
    prepared_request = request.prepare()
    print(prepared_request.url)
          
    params['_pgn'] = page_number
    
    # Send GET request to eBay with the defined parameters
    response  = requests.get(url, params=params)
    html_content = response.text # Get the HTML content of the page
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
          
    # Find the next button
    next_button = soup.find('button', class_='pagination__next', type='next')
  
    # Check if the next button is disabled
    if next_button and next_button.get('aria-disabled') == 'true':
        print('No more pages pages to scrape')
        break
    else:
        # Extract items
        items = soup.find_all('div', class_='s-item__wrapper clearfix')

        # Extract Listings
        for item in items [2:]:
            title = item.find('div', class_='s-item__title').text
            price = item.find('span', class_='s-item__price').text
            link = item.find('a', class_='s-item__link')['href'].split('?')[0]
            image_url = item.find('div', class_='s-item__image-wrapper image-treatment').find('img').get('src','No image URL')

            # Define each item as a dictionary
            item_dict = {
                'Title': title,
                'Price': price,
                'Link': link,
                'Image Link': image_url
            }

            # Append the dictionary to the list
            items_list.append(item_dict)

Scraping page: 1
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc
Scraping page: 2
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc&_pgn=1
No more pages pages to scrape


In [11]:
len(items_list)

240

In [14]:
items_df = pd.DataFrame(items_list)

In [16]:
items_df

,Title,Price,Link,Image Link
0,New ListingApple iPhone 15 Plus 128GB Black - ...,$200.00,https://www.ebay.com/itm/176791420652,https://i.ebayimg.com/images/g/fRoAAOSwfF5nh3b...
1,"New ListingIPhone 14pro, 256gb, Deep Purple",$375.00,https://www.ebay.com/itm/276817964400,https://i.ebayimg.com/images/g/MTYAAOSw~89niGN...
2,New ListingApple iPhone 11 Pro - 256 GB - Gree...,$279.00,https://www.ebay.com/itm/286266593609,https://i.ebayimg.com/images/g/kr0AAOSwC-xniGA...
3,New ListingApple iPhone 14 - 128 GB - Blue (T-...,$290.95,https://www.ebay.com/itm/126885317583,https://i.ebayimg.com/images/g/4AsAAOSwMrJnhcV...
4,New ListingApple iPhone 14 - 128 GB - Black (U...,$389.00,https://www.ebay.com/itm/196951283539,https://i.ebayimg.com/images/g/CCoAAOSwSC5niE0...
...,...,...,...,...
235,Apple iPhone 12 - A2172 - 64GB - White - Unloc...,$209.00,https://www.ebay.com/itm/387809005350,https://i.ebayimg.com/images/g/X2oAAOSwSjxngFS...
236,Apple iPhone 13 128GB - Red (Unlocked),$205.00,https://www.ebay.com/itm/167243596406,https://i.ebayimg.com/images/g/1-AAAOSwDptngCv...
237,Apple iPhone 15 - 128GB - T-Mobile (Read Descr...,$265.00,https://www.ebay.com/itm/286263076442,https://i.ebayimg.com/images/g/jEgAAOSw6ShngF3...
238,Apple iPhone 14 Pro - 128 GB - Black (Unlocked),$300.00,https://www.ebay.com/itm/335764200894,https://i.ebayimg.com/images/g/xrgAAOSwS7hngEr...
